In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
#Lecture du dictionnaire d'alignement
dictNettoyage=pd.read_excel('data/Dict3-Ancien.xlsx',sheet_name=1)

In [3]:
#Lecture des 3 fichiers données
fichier1=pd.read_excel('data/fichier1.xlsx')
fichier2=pd.read_excel('data/fichier2.xlsx')
fichier3=pd.read_excel('data/fichier3.xlsx')

In [4]:
#Suppresion des lignes de description
fichier1.drop(axis=0,inplace=True,index=0)
fichier2.drop(axis=0,inplace=True,index=0)
fichier2.drop(axis=0,inplace=True,index=1)
fichier3.drop(axis=0,inplace=True,index=0)
fichier3.drop(axis=0,inplace=True,index=1)

In [5]:
#Correction d'une imprecision au niveau du fichier 3 pour récupérer les moyennes de bac en mathématiques et physique (Car le fichier 3 stocke les notes des bacs en format du nouveau bac et les fichiers 1 et 2 contiennent des notes au format de l'ancien régime de bac)
colonnesNotes=[['Baccalauréat.7','Baccalauréat.8'],['Baccalauréat.9','Baccalauréat.10'],['Baccalauréat.11','Baccalauréat.12']]
matieresNotes=["Mathématiques","Physique-chimie"]
res={}
for m in matieresNotes:
    res[m]=[]
    for col in colonnesNotes:
        nom=col[0]
        note=col[1]
        rows=fichier3[fichier3[col[0]]==m][col[1]]
        for i in rows.index:
            res[m].append([i,note])
colMath='Baccalauréat.14'
colPhys='Baccalauréat.15'
for key in res.keys():
    for couple in res[key]:
        i=couple[0]
        col=couple[1]
        if key == "Mathématiques":
            fichier3.loc[i,colMath]=fichier3.loc[i,col]
        else:
            fichier3.loc[i,colPhys]=fichier3.loc[i,col]

In [6]:
#Suppression des anciennes colonnes
fichier3.drop(columns=[x for l in colonnesNotes for x in l ],inplace=True)

In [7]:
#Pour tester
fichier3.to_excel("data/ANOM3.xlsx")

In [8]:
#Fusion des 3 fichiers
n=len(fichier1)+len(fichier2)+len(fichier3)
#Pour un gain en performance, on a choisi d'utiliser un dictionnaire comme structure de données pour que les operations de fusions des 3 fichiers ne coutent pas trop chérs
hashMap={}

for row in dictNettoyage.itertuples():
    colFinal=row[1]
    colFichier1=row[2]
    colFichier2=row[3]
    colFichier3=row[4]
    if colFinal=="0":
        continue
    hashMap[colFinal]=[]# initialisation de la colonne avec une liste vide
    if colFichier1 in fichier1.columns:
        hashMap[colFinal].append(fichier1[colFichier1].astype(str)) # On ajoute la colonne tel qu'elle est mais convertie en type str, car n'importe quel type peut se convertir en str
    elif not colFichier1 in fichier1.columns:
        nan_arr=np.empty(len(fichier1))
        nan_arr[:]=np.nan
        hashMap[colFinal].append(nan_arr) # Sinon On ajoute une colonne vide du même taille que le fichier concérné
    if colFichier2 in fichier2.columns:# De même pour le fichier 2
        hashMap[colFinal].append(fichier2[colFichier2].astype(str))
    if not colFichier2 in fichier2.columns :
        nan_arr=np.empty(len(fichier2))
        nan_arr[:]=np.nan
        hashMap[colFinal].append(nan_arr)
    if colFichier3 in fichier3.columns:# De même pour le fichier 3
        hashMap[colFinal].append(fichier3[colFichier3].astype(str))
    if not colFichier3 in fichier3.columns :
        nan_arr=np.empty(len(fichier3))
        nan_arr[:]=np.nan
        hashMap[colFinal].append(nan_arr)
    #Jusqu'ici on aura une liste de 3 éléments où chaque élément contient les valeurs qui viennent de chaque fichier respectives
    hashMap[colFinal]=[x for l in hashMap[colFinal] for x in l] #Donc à la fin on "unpack" ou décompresse la liste

In [9]:
#On implémente une fonction qui parcours tout notre dictionnaire et nous retourne un autre dictionnaire qui contient les types de valeurs de chaque colonnes
def compterTypes(data):
    allTypes={}
    float_or_int_regex = r'^-?\d+(\.\d+)?$' #Pour chercher les valeurs numériques
    bool_regex = r'^(True|False)$' #Pour chercher les valeurs booléens
    for col in data.keys():
        valeurs_uniques=np.unique(data[col]) # On obtient juste les valeurs qui compose la colonne
        allTypes[col]=set() # Pour stocker les types on utilise un set pour qu'on aura pas des duplicats
        for val in valeurs_uniques:
            if val == 'nan' or val=="nan" or val==np.nan:
                continue
            if re.match(float_or_int_regex, str(val)):
                allTypes[col].add("numeric")
            elif re.match(bool_regex, str(val)):
                allTypes[col].add("bool")
            else :
                allTypes[col].add("string")
    return allTypes

In [10]:
#On implemente une fonction qui ne retourne une liste des colonnes qu'on doit modifier pour qu'ils soient de termes homogénes 
def compterAModifiers(data):
    allTypes=compterTypes(data)
    aModifier=[]
    for key in allTypes.keys():
        if len(allTypes[key])>1:
            aModifier.append(key)
    return aModifier

In [11]:
#On implémente une fonction pour nous afficher les colonnes qui sont hétérogénes et contiennes des valeur booléens ainsi que des chaines de charactéres ou bien booléens ainsi que des numériques
def compterAIntervenir(data):
    aModifier=compterAModifiers(data)
    allTypes=compterTypes(data)
    for key in aModifier:
        if len(allTypes[key])>1 and allTypes[key]!={'string','numeric'}:
            print(key,allTypes[key],np.unique(data[key]))

In [12]:
compterAIntervenir(hashMap)#Affichage des colonnes qui necessicitent une intervention manuelle

Genre {'bool', 'string'} ['False' 'True' 'f' 'm' 'o']
Experience_entreprise_1 {'bool', 'string'} ['False' 'OUI' 'True' 'nan' 'non' 'oui']
Experience_entreprise_2 {'bool', 'string'} ['False' 'OUI' 'True' 'nan' 'non' 'oui']
Experience_entreprise_3 {'bool', 'string'} ['False' 'OUI' 'True' 'nan' 'non' 'oui']


In [13]:
#Correction de la colonne 'Genre' dans le dictionnaire fusioné
listGenre=hashMap['Genre']
for i in range(len(listGenre)):
    if listGenre[i]=='True' or listGenre[i]=='m':
        listGenre[i]=1
    elif listGenre[i]=='False' or listGenre[i]=='f':
        listGenre[i]=0
    elif listGenre[i]=='o':
        listGenre[i]=2
hashMap['Genre']=listGenre

In [14]:
#Vérification de la correction
compterAIntervenir(hashMap)

Experience_entreprise_1 {'bool', 'string'} ['False' 'OUI' 'True' 'nan' 'non' 'oui']
Experience_entreprise_2 {'bool', 'string'} ['False' 'OUI' 'True' 'nan' 'non' 'oui']
Experience_entreprise_3 {'bool', 'string'} ['False' 'OUI' 'True' 'nan' 'non' 'oui']


In [15]:
#Pour Test
np.unique(hashMap['Boursier'])

array(['False', 'True'], dtype='<U5')

In [16]:
#On implémente une fonction pour corriger une colonne d'experience
def reparerExperience(l):
    n = len(l)
    for i in range(n):
        if l[i]=='OUI' or l[i]=='oui':
            l[i]='True'
        elif l[i]=='non':
            l[i]='False'
    return l

In [17]:
#On fait un parcours des colonnes experiences et on les corrige tous
colonnes=['Experience_entreprise_1','Experience_entreprise_2','Experience_entreprise_3']
for col in colonnes:
    hashMap[col]=reparerExperience(hashMap[col])

In [18]:
#Vérification des corrections
compterAIntervenir(hashMap)

In [19]:
#Remplacer les valeurs boléens qui sont stockés comme étant des chaines de charactéres en des vrais booléens
allTypes=compterTypes(hashMap)
for key in allTypes.keys():
    if allTypes[key]=={'bool'}:
      for i in range(len(hashMap[key])):
        if hashMap[key][i]=='True':
          hashMap[key][i]=True
        elif hashMap[key][i]=='False':
          hashMap[key][i]=False

In [20]:
#Conversion du dictionnaire en DataFrame
dfTotal=pd.DataFrame(hashMap)

In [21]:
#On implémente une fonction pour nettoyer les nom des villes pour que ce soit facile à comparer avec les villes dans les autres bases de données
def clean_ville(ville):
    return re.sub(r'[^a-zA-Z]', '', str(ville)).strip().lower()
#On appelle cette fonction sur la colonne Ville_habitation
dfTotal['Ville_habitation'] = dfTotal['Ville_habitation'].apply(lambda x: clean_ville(x))

In [22]:
#On charge la base des codes postales qui contient tous les villes de la france
cities=pd.read_csv('data/cities-Grand.csv',encoding='latin-1',sep=';')

In [23]:
#On change le nom de la colonne sur la base cities pour que ça soit le même 
cities = cities.rename(columns={'Nom_de_la_commune': 'Ville_habitation'})
cities['Ville_habitation'] = cities['Ville_habitation'].apply(lambda x: clean_ville(x))
ville_to_code_postal = dict(zip(cities['Ville_habitation'], cities['Code_postal']))

for index, row in dfTotal.iterrows():
    if pd.isnull(row['Code_Postale_Habitation']) or row['Code_Postale_Habitation'] == '':  # Vérifier si la valeur est nulle ou vide
        ville = clean_ville(row['Ville_habitation'])
        if ville in ville_to_code_postal:
          dfTotal.loc[index, 'Code_Postale_Habitation'] = ville_to_code_postal[ville]
        elif clean_ville(dfTotal.loc[index, 'Ville_annee_N']) in ville_to_code_postal:
          dfTotal.loc[index, 'Code_Postale_Habitation'] = ville_to_code_postal[clean_ville(dfTotal.loc[index, 'Ville_annee_N'])]

In [24]:
#Remplacer les valeurs 'nan' par des vrais nan
pd.set_option('future.no_silent_downcasting', True)
dfTotal = dfTotal.replace('nan', np.nan)

In [25]:
#Charger la base de données des villes avec les longitude/latitude
cities_Lat_Long = pd.read_csv('data/cities.csv')

In [26]:
cities_Lat_Long['zip_code']=cities_Lat_Long['zip_code'].astype(str)

In [27]:
#Inserer les nouvelles valeurs
dfTotal.insert(0, 'Latitude_ville_habitation', dfTotal['Code_Postale_Habitation'].apply(lambda x: cities_Lat_Long[cities_Lat_Long['zip_code'] == str(x)]['latitude'].values[0] if str(x) in cities_Lat_Long['zip_code'].values else np.nan))
dfTotal.insert(0, 'Longitude_ville_habitation', dfTotal['Code_Postale_Habitation'].apply(lambda x: cities_Lat_Long[cities_Lat_Long['zip_code'] == str(x)]['longitude'].values[0] if str(x) in cities_Lat_Long['zip_code'].values else np.nan))

In [28]:
dfTotal.drop(columns=['Ville_habitation'],inplace=True)

In [29]:
#Installation du package geopy
!pip install geopy

In [30]:
from geopy.distance import geodesic

coords_albi = (43.9276, 2.1468)

def calculer_distance(row):
    if pd.notnull(row['Latitude_ville_habitation']) and pd.notnull(row['Longitude_ville_habitation']):
        coords_etudiant = (row['Latitude_ville_habitation'], row['Longitude_ville_habitation'])
        return geodesic(coords_albi, coords_etudiant).km
    return None

dfTotal['Distance_Albi_km'] = dfTotal.apply(calculer_distance, axis=1)
dfTotal.head()


,Longitude_ville_habitation,Latitude_ville_habitation,Genre,Boursier,Tiers_Temps,Code_Postale_Habitation,Dossier_Non_Valide_pour_apprentissage,Etat_dossier_inscription,Nom_dernier_Diplome_fait,bac_francais,...,Formation_Mines_ParisTech_Energetique,Formation_ENSSAT_Informatique_Multimedia_Reseaux,Formation_ENSSAT_Informatique_photonique,Formation_SIGMA_Mecanique_Genie_Industriel,Formation_Telecom_Physique_Strasbourg_Electronique_Informatique_Industrielle,Formation_Telecom_sainté_data,Formation_Telecom_sainté_image_photonique,Formation_ET0215_1,Formation_ET0215_2,Distance_Albi_km
0,2.289706,49.900410,1,True,False,80090,False,Valide,DUT Génie mécanique et productique (GMP),True,...,non,non,NaN,non,non,NaN,NaN,non,non,664.080844
1,NaN,NaN,0,False,False,NaN,False,Valide,Autre diplôme,True,...,non,non,NaN,Non convoqué,non,NaN,NaN,non,non,NaN
2,0.066151,43.234659,1,False,False,65000,False,Valide,DUT Génie mécanique et productique (GMP),True,...,non,non,NaN,non,non,NaN,NaN,non,non,184.838994
3,5.398630,43.536112,1,False,False,13540,False,Valide,DUT Génie mécanique et productique (GMP),True,...,non,non,NaN,Admissible,non,NaN,NaN,non,non,265.556518
4,1.432095,43.596038,1,False,False,31500,False,Valide,"DUT Génie chimique, génie des procédés (GCGP)",True,...,non,non,NaN,non,non,NaN,NaN,non,non,68.332735


In [31]:
dfTotal_numeric_bool = dfTotal.select_dtypes(include=[np.number,np.bool_])

In [32]:
dfTotal.to_excel("data/BDTotal-Test.xlsx")